# Imports

In [18]:
import scipy
from scipy.optimize import linprog
#scipy.__version__

import requests
import pandas as pd

# JSON Live Station Data

In [19]:
station_info_response = requests.get("https://gbfs.citibikenyc.com/gbfs/en/station_information.json").json()
station_info = pd.DataFrame(station_info_response['data']['stations'])[["station_id","name","lon","lat"]]

station_status_response = requests.get("https://gbfs.citibikenyc.com/gbfs/en/station_status.json").json()
station_status = pd.DataFrame(station_status_response['data']['stations'])[["station_id","num_docks_available","num_bikes_available"]]

In [20]:
station_info.head(5)

,station_id,name,lon,lat
0,72,W 52 St & 11 Ave,-73.993929,40.767272
1,79,Franklin St & W Broadway,-74.006667,40.719116
2,82,St James Pl & Pearl St,-74.000165,40.711174
3,83,Atlantic Ave & Fort Greene Pl,-73.976323,40.683826
4,116,W 17 St & 8 Ave,-74.001497,40.741776


In [21]:
station_status.head(5)

,station_id,num_docks_available,num_bikes_available
0,72,17,0
1,79,11,18
2,82,11,14
3,83,3,57
4,116,54,18


In [22]:
station_merged = pd.merge(station_status,station_info,how='left')

In [23]:
station_merged.head(5)

,station_id,num_docks_available,num_bikes_available,name,lon,lat
0,72,17,0,W 52 St & 11 Ave,-73.993929,40.767272
1,79,11,18,Franklin St & W Broadway,-74.006667,40.719116
2,82,11,14,St James Pl & Pearl St,-74.000165,40.711174
3,83,3,57,Atlantic Ave & Fort Greene Pl,-73.976323,40.683826
4,116,54,18,W 17 St & 8 Ave,-74.001497,40.741776


In [24]:
print(station_info.shape)
print(station_status.shape)
print(station_merged.shape)

(1793, 4)
(1793, 3)
(1793, 6)


# LP model with a test data_set
The LP with 1000 stations takes considerably long. The Heuristic model should be used for less optimized but a faster model.

In [37]:
df = pd.read_csv("/Users/dehaay/Desktop/BikeShare Project/experiment/allocation_mock_data.csv")
#.iloc[3:6]

In [38]:
df

,Station names,capacity %,bike count in an hour,station capacity,x cord,y cord
0,A,46%,35,76,81,34
1,B,92%,65,71,47,10
2,C,162%,68,42,79,97
3,D,78%,69,89,66,32
4,E,63%,40,64,95,69
5,F,102%,49,48,4,75
6,G,93%,65,70,52,90
7,H,75%,56,75,18,15
8,I,6%,3,47,89,98
9,J,71%,63,89,74,42


In [39]:
equilibrium_percent = df["bike count in an hour"].sum() / df["station capacity"].sum()
equilibrium_percent

0.7177097203728362

In [40]:
df["give_away_stock"] = -(round(df["station capacity"] * equilibrium_percent) - df["bike count in an hour"])

In [41]:
df

,Station names,capacity %,bike count in an hour,station capacity,x cord,y cord,give_away_stock
0,A,46%,35,76,81,34,-20.0
1,B,92%,65,71,47,10,14.0
2,C,162%,68,42,79,97,38.0
3,D,78%,69,89,66,32,5.0
4,E,63%,40,64,95,69,-6.0
5,F,102%,49,48,4,75,15.0
6,G,93%,65,70,52,90,15.0
7,H,75%,56,75,18,15,2.0
8,I,6%,3,47,89,98,-31.0
9,J,71%,63,89,74,42,-1.0


In [42]:
import itertools
cross_product = list(itertools.product(df["Station names"],df["Station names"]))
station_combinations = ["_".join([i,b]) for i,b in cross_product if i != b]

In [43]:
station_combinations[1:10]

['A_C', 'A_D', 'A_E', 'A_F', 'A_G', 'A_H', 'A_I', 'A_J', 'A_K']

In [44]:
cross_product = list(itertools.product(zip(df["Station names"],df["x cord"],df["y cord"]),zip(df["Station names"],df["x cord"],df["y cord"]) ))
x_cord_diff = [ abs(i[1]-b[1]) for i,b in cross_product if i[0] != b[0]]
y_cord_diff = [ abs(i[2]-b[2]) for i,b in cross_product if i[0] != b[0]]

In [45]:
distance_combinations = [sum(i) for i in zip(x_cord_diff, y_cord_diff )] 

In [46]:
distance_combinations[1:10]

[65, 17, 49, 118, 85, 82, 72, 15, 26]

In [47]:
len(station_combinations)

702

In [48]:
len(distance_combinations)

702

## LP formulation

Objective function

\begin{equation*}
{\begin{array}{1ll}
\small\text{Minimize} \Large\sum\limits_{i}^{A} \sum\limits_{j}^{A}S_{ij} \times x_{ij} \\
& \small\text{$S_{ij}$ is transfer realtion between station i and j where i $\neq$ j} \\[2pt]
& \small\text{A: all stations} \\[2pt]
& \small\text{x: distance between station i and j} \\[2pt]
\end{array}}
\end{equation*}



$\small\text{ A: all stations $\lambda$ }$



\begin{cases}
      \Large\sum\limits_{j}^{A} S_{ji} - S_{ij}  = D_i & D_i \leq 0 \\
       \Large\sum\limits_{j}^{A} S_{ij} - S_{ji}  = D_i  & D_i \geq 0 
\end{cases}

\begin{equation*}
{\begin{array}{1ll}
\small\text{$S_{ij}$ is transfer realtion between station i and j where i $\neq$ j} \\
\small\text{D_i: Bike demand of station i for reaching the equilibrium}
\end{array}}
\end{equation*}


In [49]:
## ============================================== ##
## Decision Variables                             ## 
## ============================================== ##
VarName = station_combinations

## ============================================== ##
## Objective function coefficients (minimization) ## 
## ============================================== ##
c = distance_combinations


In [50]:
VarName[1:10]

['A_C', 'A_D', 'A_E', 'A_F', 'A_G', 'A_H', 'A_I', 'A_J', 'A_K']

In [51]:
c[1:10]

[65, 17, 49, 118, 85, 82, 72, 15, 26]

In [52]:
## ============================================== ##
## Upper Bound constraints                        ## 
## ============================================== ##
import numpy as np
A_ub, b_ub = [], []
alpha = 0.01

In [53]:
for i in range(len(df.index)):
    df_instance = df.iloc[i]
    give_away_stock = df_instance["give_away_stock"]
    station_name = df_instance["Station names"]
    station_capacity = df_instance["station capacity"]
    current_capacity = df_instance["bike count in an hour"]
    
    temp_array = np.array([0] * len(c))
    reverse_indexes = [count for count,i in enumerate(VarName) if i.split("_")[1] == station_name ]
    regular_indexes = [count for count,i in enumerate(VarName) if i.split("_")[0] == station_name ]
    

    if give_away_stock < 0:

        temp_array[reverse_indexes] = 1
        temp_array[regular_indexes] = -1
        
        
    elif give_away_stock > 0:
        
        temp_array[reverse_indexes] = -1
        temp_array[regular_indexes] = 1
 
    else:
        temp_array[reverse_indexes] = 1
        temp_array[regular_indexes] = -1

    
    
    A_ub.append((temp_array/station_capacity) )
    b_ub.append((abs(equilibrium_percent - (current_capacity/station_capacity)) + alpha))
    
    A_ub.append( -(temp_array/station_capacity) )
    b_ub.append(-(abs(equilibrium_percent - (current_capacity/station_capacity)) - alpha))
    

In [54]:
## ============================================== ##
## Nonnegativity                                  ## 
## ============================================== ##
lbs = [0]*len(c)
ubs = [None]*len(c)
bounds = [(lb, ub) for lb, ub in zip(lbs, ubs)]

In [55]:
## ============================================== ##
## Print constraints                              ## 
## ============================================== ##
for row, b in zip(A_ub, b_ub):
    print( " + ".join([repr(ele) + "*" + vn for ele, vn in zip(row, VarName) if abs(ele) > 0.0]) + " <= " + repr(b) )


-0.013157894736842105*A_B + -0.013157894736842105*A_C + -0.013157894736842105*A_D + -0.013157894736842105*A_E + -0.013157894736842105*A_F + -0.013157894736842105*A_G + -0.013157894736842105*A_H + -0.013157894736842105*A_I + -0.013157894736842105*A_J + -0.013157894736842105*A_K + -0.013157894736842105*A_L + -0.013157894736842105*A_M + -0.013157894736842105*A_N + -0.013157894736842105*A_NCDP + -0.013157894736842105*A_QEYJ + -0.013157894736842105*A_RTIH + -0.013157894736842105*A_UDXY + -0.013157894736842105*A_CVRP + -0.013157894736842105*A_GETX + -0.013157894736842105*A_RYPR + -0.013157894736842105*A_DLED + -0.013157894736842105*A_HBWL + -0.013157894736842105*A_LJYL + -0.013157894736842105*A_CGHU + -0.013157894736842105*A_YEOD + -0.013157894736842105*A_LXZI + 0.013157894736842105*B_A + 0.013157894736842105*C_A + 0.013157894736842105*D_A + 0.013157894736842105*E_A + 0.013157894736842105*F_A + 0.013157894736842105*G_A + 0.013157894736842105*H_A + 0.013157894736842105*I_A + 0.013157894736842

In [56]:
## ============================================== ##
## Print optimization result                      ## 
## ============================================== ##
res = linprog(c, A_ub = A_ub, b_ub = b_ub, bounds = bounds, method = 'highs')
print(res.message)
print("Minimized objective function value := ", res.fun)
print("Optimal solution:")
for x, vn in zip(res.x, VarName):
    print(vn, " := ", x)

Optimization terminated successfully.
Minimized objective function value :=  5611.6345006657775
Optimal solution:
A_B  :=  0.0
A_C  :=  0.0
A_D  :=  0.0
A_E  :=  0.0
A_F  :=  0.0
A_G  :=  0.0
A_H  :=  0.0
A_I  :=  0.0
A_J  :=  0.0
A_K  :=  0.0
A_L  :=  0.0
A_M  :=  0.0
A_N  :=  0.0
A_NCDP  :=  0.0
A_QEYJ  :=  0.0
A_RTIH  :=  0.0
A_UDXY  :=  0.0
A_CVRP  :=  0.0
A_GETX  :=  0.0
A_RYPR  :=  0.0
A_DLED  :=  0.0
A_HBWL  :=  0.0
A_LJYL  :=  0.0
A_CGHU  :=  0.0
A_YEOD  :=  0.0
A_LXZI  :=  0.0
B_A  :=  0.0
B_C  :=  0.0
B_D  :=  0.0
B_E  :=  0.0
B_F  :=  0.0
B_G  :=  0.0
B_H  :=  0.0
B_I  :=  0.0
B_J  :=  0.0
B_K  :=  0.0
B_L  :=  0.0
B_M  :=  0.0
B_N  :=  0.0
B_NCDP  :=  0.0
B_QEYJ  :=  0.0
B_RTIH  :=  0.0
B_UDXY  :=  0.0
B_CVRP  :=  0.0
B_GETX  :=  0.0
B_RYPR  :=  0.0
B_DLED  :=  0.0
B_HBWL  :=  0.0
B_LJYL  :=  0.0
B_CGHU  :=  13.332609853528629
B_YEOD  :=  0.0
B_LXZI  :=  0.0
C_A  :=  0.0
C_B  :=  0.0
C_D  :=  0.0
C_E  :=  0.0
C_F  :=  0.0
C_G  :=  0.0
C_H  :=  0.0
C_I  :=  30.26235685752330

# Allocation Report

$S_1\_S_2 = N$ is read transfer N bikes from $S_1$ to $S_2 $

In [57]:
[[var,round(value)] for value,var in zip(res.x, VarName) if value > 0]

[['B_CGHU', 13.332609853528629],
 ['C_I', 30.262356857523304],
 ['C_NCDP', 8.013834886817579],
 ['D_A', 7.9774700399467475],
 ['F_G', 6.418375499334226],
 ['F_RTIH', 2.7249400798934706],
 ['F_GETX', 3.1844873501997335],
 ['F_YEOD', 1.7421304926764296],
 ['G_NCDP', 27.81813581890812],
 ['H_RTIH', 2.9217709720372875],
 ['K_A', 7.51573901464714],
 ['N_CGHU', 22.342223701731026],
 ['QEYJ_GETX', 3.7084820239680436],
 ['UDXY_M', 0.9807723035952071],
 ['RYPR_D', 3.74363515312917],
 ['RYPR_M', 21.40584553928095],
 ['RYPR_LXZI', 19.74213049267643],
 ['DLED_E', 5.293422103861517],
 ['DLED_G', 7.251478029294278],
 ['DLED_CVRP', 6.262356857523299],
 ['HBWL_M', 3.6455792276964067],
 ['LJYL_A', 3.292729693741665],
 ['LJYL_J', 0.25412782956060614],
 ['LJYL_L', 9.949294274300932],
 ['LJYL_CGHU', 2.034007989347532]]

In [58]:
[[var,round(value)] for value,var in zip(res.x, VarName) if value > 0]

[['B_CGHU', 13],
 ['C_I', 30],
 ['C_NCDP', 8],
 ['D_A', 8],
 ['F_G', 6],
 ['F_RTIH', 3],
 ['F_GETX', 3],
 ['F_YEOD', 2],
 ['G_NCDP', 28],
 ['H_RTIH', 3],
 ['K_A', 8],
 ['N_CGHU', 22],
 ['QEYJ_GETX', 4],
 ['UDXY_M', 1],
 ['RYPR_D', 4],
 ['RYPR_M', 21],
 ['RYPR_LXZI', 20],
 ['DLED_E', 5],
 ['DLED_G', 7],
 ['DLED_CVRP', 6],
 ['HBWL_M', 4],
 ['LJYL_A', 3],
 ['LJYL_J', 0],
 ['LJYL_L', 10],
 ['LJYL_CGHU', 2]]